Импортируем библеотеки для работы с данными и считаем данные.

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from My_SVM import My_SVM

In [2]:
data = pd.read_csv("df_train.csv")
test = pd.read_csv("df_test.csv")
df = data.drop(["author", "class", "Unnamed: 0"], axis=1)
Y = data["author"]

Задаем целевую переменную и находим 50 лучших признаков основываясь на дисперсионный анализ, так же убираем те признаки у которых дисперсия 0, так как они не дают никакой информации и из-за них дисперсионный анализ выводит предупреждение

In [3]:
selector = VarianceThreshold()
df_new = selector.fit_transform(X=df)
selector = SelectKBest(f_classif, k=50)
X = selector.fit_transform(df_new, Y)
selected_ind = selector.get_support(indices=True)
print(list(df.columns[selected_ind]))

['и', 'социальный', ':', 'общество', 'мочь', '"', 'как', 'который', '-', 'это', 'быть', ')', '(', 'человек', 'не', 'различный', 'так', '–', 'то', 'также', 'например', 'роль', 'при', 'взаимодействие', 'друг', 'влиять', 'культурный', 'далее', 'важный', 'включать', 'влияние', 'тот', 'часто', 'только', 'понимание', 'общность', 'все', 'ключевой', 'аспект', 'же', 'взаимодействовать', 'способствовать', 'включая', 'контекст', 'рис', 'иной', 'сам', 'еще', 'отражать', '“Социальный']


Теперь поделим наши данные на обучающую и валидационную выборки

In [4]:
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.33, shuffle=True, stratify=Y)

Построим несколько моделей с разными параметрами ядра

In [20]:
model = My_SVM()
params_1 = {"C": np.logspace(-3, 3, num=7, base=10.),
            "kernel" : ["linear"],
             "decision_function_shape":["ovo"]}
params_2 = {"C": np.logspace(-3, 3, num=7, base=10.),
              "degree":[2, 3],
             "decision_function_shape":["ovo"],
               "kernel":["poly"]}
params_3 = {"C": np.logspace(-3, 3, num=7, base=10.),
               'gamma': [0.001, 0.01, 0.1],
             "decision_function_shape":["ovo"],
               "kernel":["rbf"]}
params_4 = {"C": np.logspace(-3, 3, num=7, base=10.),
               "kernel": ["sigmoid"],
               "decision_function_shape":["ovo"],
               "coef0":[0, 1, -1, 0.1, -0.1]}
multi_params = [params_1, params_2, params_3, params_4]

In [23]:
def grid_searche(params):
    gs = GridSearchCV(model, params)
    gs.fit(X_train, Y_train)
    print(f"Лучшие параметры для модели с {params['kernel']} ядром: {gs.best_params_}")
    clf = gs.best_estimator_
    print(f"Точность на валидационной выборке: {accuracy_score(clf.predict(X_val), Y_val)}\n")
    return gs.best_estimator_

In [24]:
clfs = [None]*4
for i in range(4):
    clfs[i] = grid_searche(multi_params[i])
clf1, clf2, clf3, clf4 = clfs

Лучшие параметры для модели с ['linear'] ядром: {'C': 0.1, 'decision_function_shape': 'ovo', 'kernel': 'linear'}
Точность на валидационной выборке: 0.8592964824120602

Лучшие параметры для модели с ['poly'] ядром: {'C': 10.0, 'decision_function_shape': 'ovo', 'degree': 2, 'kernel': 'poly'}
Точность на валидационной выборке: 0.8793969849246231

Лучшие параметры для модели с ['rbf'] ядром: {'C': 100.0, 'decision_function_shape': 'ovo', 'gamma': 0.001, 'kernel': 'rbf'}
Точность на валидационной выборке: 0.8492462311557789

Лучшие параметры для модели с ['sigmoid'] ядром: {'C': 1.0, 'coef0': -1, 'decision_function_shape': 'ovo', 'kernel': 'sigmoid'}
Точность на валидационной выборке: 0.8090452261306532



In [25]:
Y_pred_test_1 = clf1.predict(np.array(test[df.columns[selected_ind]]))
Y_pred_test_2 = clf2.predict(np.array(test[df.columns[selected_ind]]))
Y_pred_test_3 = clf3.predict(np.array(test[df.columns[selected_ind]]))
Y_pred_test_4 = clf4.predict(np.array(test[df.columns[selected_ind]]))
Y_pred_test_4

array(['Antoshkin', 'human', 'Antoshkin', 'human', 'gigachat', 'human',
       'Antoshkin', 'human', 'Antoshkin', 'gpt4', 'gpt4', 'gigachat',
       'human', 'Antoshkin', 'human', 'gigachat', 'gpt4', 'gpt4',
       'gigachat', 'gpt4', 'gpt4', 'Antoshkin', 'gpt4', 'human', 'human',
       'human', 'Antoshkin', 'gpt4', 'gpt4', 'gigachat', 'gpt4', 'gpt4',
       'gigachat', 'gigachat', 'gpt4', 'Antoshkin', 'gpt4', 'gigachat',
       'gpt4', 'Antoshkin', 'gigachat', 'Antoshkin', 'human', 'gpt4',
       'gpt4', 'gigachat', 'human', 'gigachat', 'gpt4', 'Antoshkin',
       'gigachat', 'gpt4', 'gigachat', 'gpt4', 'Antoshkin', 'human',
       'human', 'gpt4', 'Antoshkin', 'gigachat', 'gigachat', 'Antoshkin',
       'human', 'Antoshkin', 'gpt4', 'human', 'gpt4', 'human', 'gpt4',
       'Antoshkin', 'Antoshkin', 'gigachat', 'Antoshkin', 'human',
       'gigachat', 'Antoshkin', 'gpt4', 'Antoshkin', 'gpt4', 'human',
       'Antoshkin', 'gpt4', 'Antoshkin', 'gpt4', 'Antoshkin', 'Antoshkin',
       '

In [26]:
ans = pd.read_csv("df_test_otvety.csv")

In [27]:
print(accuracy_score(Y_pred_test_1, ans["author"]))
print(accuracy_score(Y_pred_test_2, ans["author"]))
print(accuracy_score(Y_pred_test_3, ans["author"]))
print(accuracy_score(Y_pred_test_4, ans["author"]))

0.78
0.8
0.81
0.74
